In [2]:
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer


def get_actual_answer(input: str) -> str:
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))

    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(input)}],
        user_id=user.id,
    )

2025-05-17 18:25:28 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-17 18:25:28 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
weave version 0.51.47 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-17 18:25:30 - file_cache is only supported with oauth2client<4.0.0


In [3]:
import os
import random
import json
from typing import Dict, List, Optional, Tuple, Union
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Setup OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class VietnameseUserSimulator:
    def __init__(self, persona_type: str = "random"):
        """
        Initialize the Vietnamese user simulator with a specific persona type.

        Args:
            persona_type: Type of persona to simulate ("detailed", "minimal", "impatient", "confused", or "random")
        """
        self.persona_type = (
            persona_type
            if persona_type != "random"
            else random.choice(["detailed", "minimal", "impatient", "confused"])
        )

        # Initialize persona traits using OpenAI API
        self._generate_persona()

        # Track conversation state
        self.conversation_history = []
        self.provided_contact_info = False
        self.product_interest_stated = False
        self.brand_preference_stated = False
        self.questions_stated = False
        self.ready_to_purchase = False

    def _generate_persona(self):
        """Generate a realistic Vietnamese customer persona using OpenAI API with structured output"""

        # Few-shot examples for different persona types
        persona_examples = {
            "detailed": {
                "name": "Nguyễn Minh Hải",
                "email": "minhhai88@gmail.com",
                "phone": "0903456789",
                "age": 32,
                "occupation": "Software Engineer",
                "product_type": "laptop",
                "budget": 25000000,
                "preferred_brand": "Dell",
                "technical_knowledge": "high",
                "verbosity": "high",
                "patience": "high",
                "willingness_to_share_info": "high",
                "questions": [
                    "Sản phẩm có đi kèm bảo hành mở rộng không?",
                    "Có khuyến mãi nào áp dụng hiện tại không?",
                    "Tôi có thể chọn màu sắc không?",
                    "Máy có hỗ trợ nâng cấp phần cứng trong tương lai không?",
                    "Thời lượng pin thực tế khi sử dụng lập trình và chạy máy ảo là bao lâu?",
                    "Màn hình có hỗ trợ tần số quét cao không?",
                    "Có đi kèm hệ điều hành bản quyền không?",
                ],
                "communication_style": "Likes to discuss technical details, asks many questions about specifications, compares different models, provides complete information when asked",
            },
            "minimal": {
                "name": "Trần Thị Mai",
                "email": "maimai@gmail.com",
                "phone": "0912345678",
                "age": 27,
                "occupation": "Office Worker",
                "product_type": "phone",
                "budget": 7000000,
                "preferred_brand": "Samsung",
                "technical_knowledge": "medium",
                "verbosity": "low",
                "patience": "medium",
                "willingness_to_share_info": "medium",
                "questions": [
                    "Giá đã bao gồm phụ kiện chưa?",
                    "Có được giảm giá nếu thanh toán qua ví điện tử không?",
                    "Máy có bao nhiêu màu để chọn?",
                    "Có được trả góp không?",
                ],
                "communication_style": "Provides short answers, often one word responses, focuses on price, hesitant to share personal information immediately",
            },
            "impatient": {
                "name": "Lê Văn Dũng",
                "email": "levandung75@gmail.com",
                "phone": "0865432198",
                "age": 45,
                "occupation": "Business Owner",
                "product_type": "phone",
                "budget": 30000000,
                "preferred_brand": "iPhone",
                "technical_knowledge": "medium",
                "verbosity": "medium",
                "patience": "low",
                "willingness_to_share_info": "low",
                "questions": [
                    "Giá rẻ nhất hiện tại là bao nhiêu?",
                    "Có khuyến mãi gì không? Nói nhanh giúp tôi.",
                    "Tôi mua hôm nay có được tặng phụ kiện gì không?",
                    "Bảo hành chính hãng hay bên cửa hàng?",
                    "Máy có còn nguyên seal không?",
                ],
                "communication_style": "Asks about prices quickly, shows impatience with phrases like 'nhanh lên', focuses on getting the best deal, reluctant to share contact info until convinced of value",
            },
            "confused": {
                "name": "Phạm Thanh Hà",
                "email": "hathanhpham@yahoo.com.vn",
                "phone": "0389765432",
                "age": 52,
                "occupation": "Teacher",
                "product_type": "laptop",
                "budget": 15000000,
                "preferred_brand": "HP",
                "technical_knowledge": "low",
                "verbosity": "high",
                "patience": "medium",
                "willingness_to_share_info": "medium",
                "questions": [
                    "Cái này dùng để soạn bài được không?",
                    "Nó có nhẹ không, tôi cần mang đi lại nhiều?",
                    "Bảo hành có khó làm thủ tục không?",
                    "Có cần phải mua thêm phần mềm gì nữa không?",
                    "Nếu tôi mua thì mấy ngày nhận được?",
                    "Có cài sẵn gì trong máy không?",
                ],
                "communication_style": "Often confuses technical terms, asks for clarification frequently, changes questionsring conversation, mixes up RAM and storage, repeats questions",
            },
        }
        # Create system prompt for persona generation
        system_prompt = """You are an API that generates realistic Vietnamese customer personas for an e-commerce simulator. 
Generate a detailed persona for a Vietnamese customer interested in technology products.
Your output must be VALID JSON that matches the structure of the example.
Use realistic Vietnamese names, contact information, and behavior patterns."""

        # Select a few-shot example based on persona type
        example = persona_examples[self.persona_type]
        example_json = json.dumps(example, ensure_ascii=False, indent=2)

        # Create user prompt with instructions and example
        user_prompt = f"""Generate a realistic Vietnamese customer persona with the following characteristics:
- Persona type: {self.persona_type}
- Product interest: Either phone or laptop
- Include realistic Vietnamese name, email, phone number
- Include appropriate budget in VND
- Include technical questionspropriate for the product
- Specify behavioral traits: technical_knowledge, verbosity, patience, willingness_to_share_info

Use this structure as a guide:
{example_json}

Respond ONLY with a valid JSON object matching this structure but with different realistic values.
"""

    
        # Call OpenAI API for structured persona generation
        response = client.chat.completions.create(
            model="gpt-4o",
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.8,
        )
        # Parse the response
        persona_data = json.loads(response.choices[0].message.content)
        # Set persona attributes
        self.name = persona_data.get("name", "Nguyễn Văn A")
        self.email = persona_data.get(
            "email", f"user{random.randint(100,999)}@gmail.com"
        )
        self.phone = persona_data.get(
            "phone", f"09{random.randint(10000000, 99999999)}"
        )
        self.age = persona_data.get("age", random.randint(18, 65))
        self.occupation = persona_data.get("occupation", "Unknown")
        self.product_type = persona_data.get(
            "product_type", random.choice(["phone", "laptop"])
        )
        self.budget = persona_data.get(
            "budget",
            (
                random.randint(2_000_000, 40_000_000)
                if self.product_type == "phone"
                else random.randint(10_000_000, 60_000_000)
            ),
        )
        self.preferred_brand = persona_data.get("preferred_brand", "")
        self.technical_knowledge = persona_data.get("technical_knowledge", "medium")
        self.verbosity = persona_data.get("verbosity", "medium")
        self.patience = persona_data.get("patience", "medium")
        self.willingness_to_share_info = persona_data.get(
            "willingness_to_share_info", "medium"
        )
        self.questions = persona_data.get("questions", {})
        self.communication_style = persona_data.get("communication_style", "")

    def _get_vietnamese_formality(self) -> Dict[str, str]:
        """Get Vietnamese formality based on persona"""
        # Choose pronouns based on persona
        # In Vietnamese, pronouns vary greatly and indicate social relationships
        options = [
            {"self": "Tôi", "you": "bạn"},  # Neutral/formal
            {"self": "Em", "you": "anh/chị"},  # Younger person to older
            {"self": "Mình", "you": "bạn"},  # Friendly/informal
        ]
        return random.choice(options)

    def generate_response(self, agent_message: str) -> str:
        """
        Generate a user response to the workflow agent's message.

        Args:
            agent_message: The message from the workflow agent

        Returns:
            The simulated user's response
        """
        # Add agent message to conversation history
        self.conversation_history.append(
            {"role": "assistant", "content": agent_message}
        )

        # Determine appropriate response based on conversation state and persona
        system_prompt = self._create_system_prompt()

        # Create few-shot examples based on persona type
        few_shot_examples = self._get_few_shot_examples()

        # Create messages for the API call
        messages = [
            {"role": "system", "content": system_prompt},
            *few_shot_examples,
            *self.conversation_history,
        ]

        # Get response from OpenAI
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Or another appropriate model
            messages=messages,
            temperature=0.9 if self.persona_type == "confused" else 0.7,
            max_tokens=150,
        )

        user_response = response.choices[0].message.content

        # Update conversation history with user's response
        self.conversation_history.append({"role": "user", "content": user_response})

        # Update conversation state based on response
        self._update_conversation_state(user_response)

        return user_response

    def _get_few_shot_examples(self) -> List[Dict[str, str]]:
        """Get few-shot examples for the specific persona type"""
        examples = {
            "detailed": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Tôi đang cần mua một chiếc laptop mới để làm việc. Tôi làm trong lĩnh vực thiết kế đồ họa nên cần một chiếc máy có cấu hình mạnh.",
                },
                {
                    "role": "assistant",
                    "content": "Dạ vâng, tôi rất vui được tư vấn cho anh/chị. Đối với công việc thiết kế đồ họa, anh/chị quan tâm đến thương hiệu nào cụ thể không ạ?",
                },
                {
                    "role": "user",
                    "content": "Tôi thích các dòng máy của Dell hoặc ASUS. Tôi cần một máy có card đồ họa tốt, RAM ít nhất 16GB và bộ xử lý mạnh. Màn hình cũng cần có độ phân giải cao và độ phủ màu tốt. Budget của tôi khoảng 30 triệu.",
                },
            ],
            "minimal": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {"role": "user", "content": "Tìm điện thoại"},
                {
                    "role": "assistant",
                    "content": "Dạ vâng, bạn đang quan tâm đến điện thoại hãng nào ạ? Và bạn có yêu cầu gì về cấu hình và giá cả không ạ?",
                },
                {"role": "user", "content": "Samsung. Dưới 10tr"},
            ],
            "impatient": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Cần mua điện thoại gấp. Có iPhone 15 không?",
                },
                {
                    "role": "assistant",
                    "content": "Dạ chào anh/chị. Hiện tại cửa hàng có iPhone 15 nhiều phiên bản. Anh/chị quan tâm đến mẫu nào cụ thể ạ? iPhone 15, 15 Plus, 15 Pro hay 15 Pro Max?",
                },
                {
                    "role": "user",
                    "content": "Giá bao nhiêu? Nhanh lên, tôi không có nhiều thời gian. 15 Pro Max 256GB giá bao nhiêu?",
                },
            ],
            "confused": [
                {
                    "role": "assistant",
                    "content": "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?",
                },
                {
                    "role": "user",
                    "content": "Tôi muốn mua laptop nhưng không biết nên mua loại nào. Có nhiều RAM là tốt phải không?",
                },
                {
                    "role": "assistant",
                    "content": "Dạ vâng, RAM là một yếu tố quan trọng. Anh/chị dự định sử dụng laptop cho công việc gì ạ?",
                },
                {
                    "role": "user",
                    "content": "À tôi chỉ dùng để lướt web thôi. Nhưng đôi khi tôi cũng chỉnh sửa ảnh. RAM là bộ nhớ trong đúng không? Hay là ổ cứng nhỉ? Tôi cần ổ cứng lớn để lưu nhiều ảnh. Mà máy càng nhẹ càng tốt.",
                },
            ],
        }

        return examples.get(self.persona_type, examples["detailed"])


    def _create_system_prompt(self) -> str:
        """Create a system prompt for the API based on persona and conversation state"""
        formality = self._get_vietnamese_formality()

        prompt = f"""You are simulating a Vietnamese user who is interested in purchasing {"a phone" if self.product_type == "phone" else "a laptop"}.

PERSONA DETAILS:
- Type: {self.persona_type} 
- Name: {self.name}
- Product interest: {self.product_type}
- Preferred brand: {self.preferred_brand}
- Budget: {self.budget:,} VND (about {round(self.budget/23000, 2)} USD)
- Technical knowledge: {self.technical_knowledge}
- Patience level: {self.patience}
- Verbosity: {self.verbosity}
- Willingness to share contact info: {self.willingness_to_share_info}
- Vietnamese pronouns: refers to self as '{formality["self"]}', refers to agent as '{formality["you"]}'

Your responses should be primarily in Vietnamese with occasional English words (as is common in Vietnamese tech conversations). 
Use Vietnamese chat style and abbreviations when appropriate:
- "ko" for "không" (no)
- "ok" or "oki" for agreement
- "đc" for "được" (can/ok)
- "khoảng" abbreviated as "khoang" 
- Numbers + "tr" for millions (e.g., "15 tr" = 15 million VND)

PERSONA BEHAVIOR GUIDELINES:
"""

        # Add persona-specific behavior guidelines
        if self.persona_type == "detailed":
            prompt += """
- Provide detailed information about your needs and preferences
- Ask technical questions about the products
- Respond directly to questions and provide thorough information when asked
- Be polite and patient
- Willing to share contact information when it makes sense
"""
        elif self.persona_type == "minimal":
            prompt += """
- Keep responses very short, often just a few words
- Answer questions directly with minimal elaboration
- Sometimes use single word answers or short Vietnamese phrases
- Prefer simple language and avoid technical details
- Hesitant to share contact information immediately but will if pressed
"""
        elif self.persona_type == "impatient":
            prompt += """
- Show signs of impatience with phrases like "nhanh lên", "mau", etc.
- Ask about prices early and often
- Focus heavily on getting the best deal
- May sound abrupt or slightly demanding
- Reluctant to share contact information until convinced of good value
- Might try to negotiate or ask about discounts
"""
        elif self.persona_type == "confused":
            prompt += """
- Ask for clarification frequently
- Sometimes misunderstand basic product specifications
- Change your mind about questionsd-conversation
- Ask repeated questions about the same topics
- Get details wrong or confused (like mixing up RAM and storage)
- Somewhat hesitant to share contact information due to confusion
"""

        # Add information about product questions
        prompt += f"\nPRODUCT questions:\n"
        for i, value in enumerate(self.questions):
            prompt += f"- {i}: {value}\n"

        # Conversation state-specific guidance
        prompt += "\nCONVERSATION STATE GUIDELINES:\n"

        if not self.product_interest_stated:
            prompt += (
                "- When asked about product type, indicate your interest in "
                + self.product_type
                + "\n"
            )

        if self.product_interest_stated and not self.brand_preference_stated:
            prompt += f"- If asked about brand preference, mention {self.preferred_brand} or say you're open to suggestions\n"

        if not self.questions_stated:
            prompt += "- Be ready to mention your questions when the conversation moves to specific product features\n"

        if not self.provided_contact_info:
            if self.willingness_to_share_info == "high":
                prompt += "- Provide your contact info (email and phone) when asked or when it seems appropriate\n"
            elif self.willingness_to_share_info == "medium":
                prompt += "- Initially hesitate but eventually provide contact info when asked repeatedly\n"
            else:
                prompt += "- Be very reluctant to share contact info; may need significant convincing\n"

        prompt += """
IMPORTANT:
- Respond ONLY as the user would, do not include explanations or meta-commentary
- Your response should look like a genuine Vietnamese customer chat message
- Do not use quotation marks around your response
- Sometimes include typos as would be natural in chat
- If the agent asks for your contact information and you're ready to provide it, give your name, email, and phone number
"""

        return prompt

    def _update_conversation_state(self, response: str) -> None:
        """Update conversation state based on the user's response"""
        response_lower = response.lower()

        # Check if product interest is stated
        if not self.product_interest_stated:
            if (
                "điện thoại" in response_lower
                or "phone" in response_lower
                or "smartphone" in response_lower
            ):
                self.product_interest_stated = True
                self.product_type = "phone"
            elif (
                "laptop" in response_lower
                or "máy tính" in response_lower
                or "notebook" in response_lower
            ):
                self.product_interest_stated = True
                self.product_type = "laptop"

        # Check if brand preference is stated
        if self.product_interest_stated and not self.brand_preference_stated:
            brands = [
                "samsung",
                "iphone",
                "apple",
                "oppo",
                "xiaomi",
                "vivo",
                "realme",
                "dell",
                "hp",
                "asus",
                "acer",
                "lenovo",
                "macbook",
            ]
            for brand in brands:
                if brand in response_lower:
                    self.brand_preference_stated = True
                    self.preferred_brand = brand
                    break

        # Check if contact info is provided
        if not self.provided_contact_info:
            # Check for email patterns
            if "@" in response_lower and (
                ".com" in response_lower or ".vn" in response_lower
            ):
                self.provided_contact_info = True

            # Check for phone number patterns
            phone_patterns = ["09", "08", "07", "03", "05", "+84"]
            for pattern in phone_patterns:
                if pattern in response_lower and any(
                    c.isdigit() for c in response_lower
                ):
                    self.provided_contact_info = True
                    break

        # Check if questions are stated
        if not self.questions_stated:
            requirement_keywords = [
                "ram",
                "bộ nhớ",
                "camera",
                "pin",
                "màn hình",
                "screen",
                "processor",
                "cpu",
                "storage",
                "ổ cứng",
                "ssd",
                "hdd",
            ]
            for keyword in requirement_keywords:
                if keyword in response_lower:
                    self.questions_stated = True
                    break


def simulate_conversation(workflow_agent, user_simulator, max_turns=10):
    """
    Simulate a conversation between the workflow agent and user simulator

    Args:
        workflow_agent: Function that takes a user message and returns an agent response
        user_simulator: VietnameseUserSimulator instance
        max_turns: Maximum number of conversation turns

    Returns:
        List of conversation turns
    """
    conversation = []

    # Start conversation with a greeting from the workflow agent
    agent_message = "Xin chào! Tôi có thể giúp gì cho bạn hôm nay?"
    user_message = user_simulator.generate_response(agent_message)
    conversation.append({"role": "user", "content": user_message})

    # Continue conversation for up to max_turns
    for _ in range(max_turns - 1):
        # Get agent response
        agent_message = workflow_agent(user_message)
        conversation.append({"role": "assistant", "content": agent_message})

        # Check if conversation should end
        if (
            "liên hệ" in agent_message.lower()
            and "cảm ơn" in agent_message.lower()
            and user_simulator.provided_contact_info
        ):
            # Agent has thanked user for contact info and promised follow-up
            break

        # Get user response
        user_message = user_simulator.generate_response(agent_message)
        conversation.append({"role": "user", "content": user_message})

    return conversation

# Example usage:
def example_workflow_agent(user_message: str, user, thread) -> str:
    """
    Example workflow agent implementation (placeholder for your actual agent)
    This is just to demonstrate how to use the user simulator
    """
    # This is where you would connect to your actual workflow agent
    # For demonstration purposes, we'll use a simple rule-based response

    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(user_message)}],
        user_id=user.id,
    )


# Run simulation
if __name__ == "__main__":
    # Create different persona types for demonstration
    personas = ["detailed"]

    for persona in personas:
        user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
        thread = create_thread(CreateThreadModel(id = uuid4(),
            user_id=user.id, name=user.user_name))
        print(f"\n{'='*50}")
        print(f"SIMULATING CONVERSATION WITH {persona.upper()} PERSONA")
        print(f"user_id: {user.id}")
        print(f"thread_id: {thread.id}")
        print(f"{'='*50}\n")

        user_sim = VietnameseUserSimulator(persona_type=persona)
        conversation = simulate_conversation(lambda msg: example_workflow_agent(msg, user, thread), user_sim)

        print(f"Persona: {persona}")
        print(f"Name: {user_sim.name}")
        print(f"Product: {user_sim.product_type}")
        print(f"Brand: {user_sim.preferred_brand}")
        print(f"Budget: {user_sim.budget:,} VND")
        print("\nCONVERSATION:")

        for turn in conversation:
            if turn["role"] == "assistant":
                print(f"\nAgent: {turn['content']}")
            else:
                print(f"\nUser: {turn['content']}")

        print(f"\nContact info provided: {user_sim.provided_contact_info}")
        print(f"Product interest stated: {user_sim.product_interest_stated}")
        print(f"Brand preference stated: {user_sim.brand_preference_stated}")
        print(f"questions stated: {user_sim.questions_stated}")


SIMULATING CONVERSATION WITH DETAILED PERSONA
user_id: 2ad9104c-420e-4bb5-a9d2-e774a8cbee13
thread_id: 730f4741-1206-4133-ba64-2e7fdac6bf69

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196ddfd-f4db-7a60-86b6-0cd17f41aa76
2025-05-17 18:25:34 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196ddfe-062f-7741-b1bb-f490941c1f49
2025-05-17 18:25:38 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196ddfe-166a-7861-921d-c3f3cedaf0ac
2025-05-17 18:25:40 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.LAPTOP: 'laptop'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=<ProductType.LAP

KeyboardInterrupt: 

In [ ]:
from service.laptop import search, LaptopFilter

filter = LaptopFilter(min_price=1000000)
search(filter)

2025-05-17 18:08:42 - Loaded .env file
2025-05-17 18:08:45 - Warning: Traces will not be logged. Call weave.init to log your traces to a project.
 (subsequent messages of this type will be suppressed)


[LaptopModel(id='431997138342', data={'code': '431997138342', 'name': 'Asus Vivobook E1404FA-NK186W R5 7520U', 'skus': [{'sku': '00872250', 'name': 'Laptop Asus Vivobook E1404FA-NK186W R5 7520U/16GB/512GB/14" FHD/Win11', 'slug': 'may-tinh-xach-tay/asus-vivobook-e1404fa-nk186w-r5-7520u?sku=00872250', 'type': 'Normal', 'image': 'https://cdn2.fptshop.com.vn/unsafe/2023_4_11_638168280594574722_asus-vivobook-e1404fa-nk186w-r5-7520u-den-5.jpg', 'badges': [{'id': 6, 'name': 'Độc quyền', 'image': 'https://cdn2.fptshop.com.vn/svg/Loai_Doc_quyen_Mau_do_No_c3b0a3b6f4.svg'}, {'id': 46, 'name': 'Giá chạm đáy', 'image': 'https://cdn2.fptshop.com.vn/svg/Loai_gia_cham_day_Mau_do_yes_0d2fe37e9f.svg'}], 'labels': [{'code': 'zero', 'name': 'Trả góp 0%', 'image': '/', 'discountAmount': 0}], 'refCode': '00872250', 'variants': [{'code': '#000000', 'value': 'Đen', 'displayOrder': 0, 'displayValue': 'Đen', 'propertyName': 'color'}], 'promotions': [{'image': {'src': 'https://s3-sgn09.fptcloud.com/ict-k8s-promo